In [1]:
import oat_python as oat

import plotly.graph_objects as go
import numpy as np
import sklearn
import numpy as np
import plotly.express as px

/Users/floyd/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Load the point cloud

In [3]:
#   LOAD THE 1000 POINT DRAGON CLOUD
cloud   =   np.loadtxt("https://raw.githubusercontent.com/OpenAppliedTopology/oat_jupyter/refs/heads/main/source_data/dragon_vrip.ply.txt_1000_.txt")
cloud   =   cloud[:,[0,2,1]]

#   PLOT THE POINT CLOUD
trace   =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker = dict(opacity=1.0, size=3, color=cloud[:,1], colorscale="Peach"))
fig     =   go.Figure(data=trace)
fig.update_layout( 
    title=dict(text="Stanford dragon, 1000 points"), 
    template="plotly_dark",
    width=1400, 
    height=1200,  
    )
fig.show()

# Compute persistent homology

We compute persistent homology by factoring the boundary matrix.  The following cell generates a sparse distance matrix and feeds it to the persistent homology solver.  The result is a factored boundary matrix.  We will extract information from this matrix in the following cells.

In [4]:
# the minimum enclosing radius; all homology vanishes above this filtration parameter
enclosing               =   oat.dissimilarity.enclosing_from_cloud(cloud)   

# distance matrix with values over enclosing + 0.0000000001 removed; adding 0.0000000001 avoids problems due to numerical error
dissimilairty_matrix    =   oat.dissimilarity.matrix_from_cloud(            
                                cloud                       =   cloud,
                                dissimilarity_max           =   enclosing + 0.0000000001,
                            )

# build and factor the boundary matrix
factored                =   oat.rust.FactoredBoundaryMatrixVr( 
                                dissimilarity_matrix        =   dissimilairty_matrix,
                                homology_dimension_max      =   1,
                            )

Full details on `FactoredBoundaryMatrixVr` can be retreived with Python's `help` function.

In [ ]:
help(oat.rust.FactoredBoundaryMatrixVr)

# Plot the persistence diagram

In [5]:
#   PLOT THE BARCODE

homology            =   factored.homology(
                            return_cycle_representatives     =   True,
                            return_bounding_chains          =   True,
                        )
fig_pd              =   oat.plot.pd( homology )
fig_pd.show()

# Plot the barcode

In [6]:
fig_barcode              =   oat.plot.barcode( barcode=homology )
fig_barcode.show()

# Inspect homology and cycle representatives

The `homology` object is a data frame

In [7]:
display(homology)

,dimension,birth,death,birth simplex,death simplex,cycle representative,cycle nnz,bounding chain,bounding nnz
id,,,,,,,,,
0,0,0.000000,0.006098,[999],"[476, 999]",simplex filtration coefficient 0 [999] ...,2,"simplex filtration coefficient 0 [476,...",1.0
1,0,0.000000,0.005790,[998],"[53, 998]",simplex filtration coefficient 0 [998] ...,2,"simplex filtration coefficient 0 [53, 9...",1.0
2,0,0.000000,0.005510,[997],"[539, 997]",simplex filtration coefficient 0 [997] ...,2,"simplex filtration coefficient 0 [539,...",3.0
3,0,0.000000,0.001022,[996],"[889, 996]",simplex filtration coefficient 0 [996] ...,2,"simplex filtration coefficient 0 [889,...",1.0
4,0,0.000000,0.008283,[995],"[936, 995]",simplex filtration coefficient 0 [995] ...,2,"simplex filtration coefficient 0 [936,...",3.0
...,...,...,...,...,...,...,...,...,...
1306,1,0.004276,0.004310,"[305, 404]","[305, 454, 581]","simplex filtration coefficient 0 [305,...",4,simplex filtration coefficient 0 ...,2.0
1307,1,0.004200,0.004236,"[6, 384]","[6, 384, 650]","simplex filtration coefficient 0 [6,...",4,simplex filtration coefficient 0 [6...,2.0
1308,1,0.004193,0.004359,"[418, 443]","[200, 387, 994]","simplex filtration coefficient 0 [418,...",6,simplex filtration coefficient 0 ...,4.0


# Inspect a cycle representative and its bounding chain

By default, terms appear in reverse filtration order (ties are broken by reverse lexicographic order)

In [8]:
homology["cycle representative"][420]

,simplex,filtration,coefficient
0,[579],0.0,1
1,[445],0.0,-1


In [9]:
homology["bounding chain"][420]

,simplex,filtration,coefficient
0,"[445, 579]",0.001726,1


# Plot a representative

In [10]:
#   FIND THE LONGEST BAR IN DIMENSION 1

def lifetime(p):
     """
     gets the lifetime of a feature; returns -infinity for features of dimension != 1
     """
     if homology["dimension"][p]!= 1:
          return -np.inf
     else:
          return homology["death"][p] - homology["birth"][p]

max( 
     range(homology.shape[0]),  # number of rows in the data frame
     key    =   lifetime,
)

1152

In [11]:


edges               =   homology["cycle representative"][1152]["simplex"].tolist() # the cycle
triangles           =   homology["bounding chain"][1152]["simplex"].tolist() # the chain that bounds the cycle
coo                 =   cloud # coo stands for coordinate oracle

traces_edge         =   [ oat.plot.edge__trace3d( edge, coo  ) for edge in edges ]
trace_triangle      =   oat.plot.triangles__trace3d( triangles, coo ) 
trace_cloud         =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker = dict(opacity=0.8, size=3, color=cloud[:,1], colorscale="Peach"))

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
trace_triangle.update(showlegend=True, legendgroup="triangles", opacity=0.5, name="Bounding chain", color="white")
for trace_number, trace in enumerate( traces_edge ):
    showlegend      =   trace_number == 0
    trace.update(showlegend=showlegend, legendgroup="edges", opacity=0.8, name="Initial cycle", line=dict(color="orange", width=10))

fig = go.Figure(data= [trace_cloud] + traces_edge + [trace_triangle] )
fig.update_layout(
        title=dict(text="Cycle representative and bounding chain"),
        template="plotly_dark",
        height=1000,
        width=1200,
    )
fig.update_layout() 
fig.show()

Suggestion: consider using `triangle__trace3d` instead of `triangles__trace3d`, as the former often gives higher quality graphics.


In [12]:
# optimize the cycle
optimal     =   factored.optimize_cycle(
                    birth_simplex                   =   homology["birth simplex"][1152], 
                    problem_type                    =   "preserve PH basis",
                )

# display the data frame that contains the solution
display(optimal)

# display the data frame for just the optimal cycle (which is contained in the larger data frame)
display(optimal["chain"]["optimal cycle"])


Finished construcing L1 optimization program.
Constraint matrix has 11601 nonzero entries.
Passing program to solver.

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 5713, num_constraints: 6648, objective: 0.19165160828520839 }


,cost,nnz,chain
type of chain,,,
initial cycle,0.278544,53,simplex filtration coefficient 0 [62...
optimal cycle,0.191652,24,simplex filtration coefficient 0 [62...
difference in bounding chains,NaN,41,simplex filtration coefficient 0 ...
difference in essential chains,NaN,0,"Empty DataFrame Columns: [simplex, filtration,..."
Ax + z - y,NaN,0,"Empty DataFrame Columns: [simplex, filtration,..."


,simplex,filtration,coefficient
0,"[620, 742]",0.010779,-1
1,"[792, 910]",0.010647,-1
2,"[367, 910]",0.010539,1
3,"[119, 552]",0.010345,1
4,"[232, 512]",0.010238,1
5,"[182, 684]",0.010232,-1
6,"[334, 681]",0.010081,-1
7,"[59, 334]",0.009962,-1
8,"[367, 612]",0.008719,-1
9,"[236, 833]",0.008416,-1


In [13]:


edges               =   optimal["chain"]["optimal cycle"]["simplex"].tolist() # the cycle
triangles           =   optimal["chain"]["difference in bounding chains"]["simplex"].tolist() # the chain that bounds the cycle
coo                 =   cloud # coo stands for coordinate oracle

traces_edge         =   [ oat.plot.edge__trace3d( edge, coo  ) for edge in edges ]
trace_triangle      =   oat.plot.triangles__trace3d( triangles, coo ) 
trace_cloud         =   go.Scatter3d(x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], mode="markers", marker = dict(opacity=0.8, size=3, color=cloud[:,1], colorscale="Peach"))

trace_cloud.update(showlegend=True, opacity=0.5, name="Point cloud")
trace_triangle.update(showlegend=True, legendgroup="triangles", opacity=0.5, name="Bounding chain", color="white")
for trace_number, trace in enumerate( traces_edge ):
    showlegend      =   trace_number == 0
    trace.update(showlegend=showlegend, legendgroup="edges", opacity=0.8, name="Initial cycle", line=dict(color="orange", width=10))

fig = go.Figure(data= [trace_cloud] + traces_edge + [trace_triangle] )
fig.update_layout(
        title=dict(text="Cycle representative and bounding chain"),
        template="plotly_dark",
        height=1000,
        width=1200,
    )
fig.update_layout() 
fig.show()

Suggestion: consider using `triangle__trace3d` instead of `triangles__trace3d`, as the former often gives higher quality graphics.
